In [ ]:
# =============================================================================
# 🎯 FinSight - Sistema de Prevenção ao Rotativo no Cartão de Crédito
# =============================================================================
# 📊 Geração de Base Sintética para Modelo Preditivo
# =============================================================================

# 🎯 OBJETIVO DO CASE: 
# Desenvolver um modelo de machine learning que antecipe quando clientes "bons" 
# (estáveis) podem entrar em rotativo no cartão de crédito devido a eventos 
# de estresse financeiro.

# 💡 PROBLEMA DE NEGÓCIO:
# Clientes historicamente bons que passam por dificuldades financeiras começam 
# a usar rotativo, gerando prejuízos para o banco e risco de inadimplência.

# 🚀 SOLUÇÃO PROPOSTA:
# Criar uma base sintética realista que simule 3 perfis de clientes e treinar 
# um modelo para identificar sinais precoces de estresse.

In [ ]:
# =============================================================================
# 📦 IMPORTAÇÕES DE BIBLIOTECAS
# =============================================================================
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# =============================================================================
# ⚙️ CONFIGURAÇÃO DA SIMULAÇÃO - PARÂMETROS GLOBAIS
# =============================================================================

N_CLIENTES = 10000     # Número de clientes para simular
N_MESES = 24           # Histórico de 24 meses (2 anos)
MES_INICIO_ESTRESSE = 18 # Mês em que o "evento de estresse" começa
PCT_ESTRESSE = 0.20    # % de clientes bons que sofrerão estresse

# 📊 Probabilidades demográficas baseadas em dados reais (Serasa & ANBIMA)
PROB_IDADE = [0.113, 0.338, 0.354, 0.195]      # [18-25, 26-40, 41-65, >65]
PROB_CLASSE = [0.24, 0.47, 0.29]               # [A/B, C, D/E]
PROB_REGIAO = [0.43, 0.26, 0.15, 0.08, 0.08]   # [Sudeste, Nordeste, Sul, Norte, Centro-Oeste]

print(f"🚀 Iniciando simulação para {N_CLIENTES} clientes em {N_MESES} meses...")

In [ ]:
# =============================================================================
# 👥 FUNÇÃO PARA CRIAR OS PERFIS ESTÁTICOS DOS CLIENTES
# =============================================================================
def criar_clientes_estaticos(n_clientes):
    """
    🎯 OBJETIVO: Criar a base inicial de clientes com seus perfis demográficos 
    e financeiros que não mudam (ou mudam pouco) ao longo do tempo.
    """
    df_clientes = pd.DataFrame(
        index=np.arange(1, n_clientes + 1),
        columns=[
            'id_cliente', 'idade_faixa', 'classe_social', 'regiao', 
            'perfil_investidor', 'limite_cartao', 'scr_divida_inicial', 
            'grupo_simulacao'
        ]
    )
    
    df_clientes['id_cliente'] = np.arange(1, n_clientes + 1)
    
    # --- Demografia (Baseado nos relatórios oficiais) ---
    df_clientes['idade_faixa'] = np.random.choice(
        ['18-25', '26-40', '41-65', '65+'], n_clientes, p=PROB_IDADE
    )
    df_clientes['classe_social'] = np.random.choice(
        ['A/B', 'C', 'D/E'], n_clientes, p=PROB_CLASSE
    )
    df_clientes['regiao'] = np.random.choice(
        ['Sudeste', 'Nordeste', 'Sul', 'Norte', 'Centro-Oeste'], n_clientes, p=PROB_REGIAO
    )
    
    # --- Perfil Financeiro (Baseado na classe social) ---
    probs_perfil = {
        'A/B': [0.36, 0.22, 0.13, 0.29],  # [Diversifica, Caderneta, Economiza, Sem Reserva]
        'C':   [0.13, 0.22, 0.15, 0.50],
        'D/E': [0.05, 0.15, 0.09, 0.71]
    }
    
    df_clientes['perfil_investidor'] = df_clientes['classe_social'].apply(
        lambda x: np.random.choice(['Diversifica', 'Caderneta', 'Economiza', 'Sem Reserva'], p=probs_perfil[x])
    )
    
    # --- Definir Limites e Dívidas Iniciais ---
    limites = {'A/B': (10000, 30000), 'C': (3000, 10000), 'D/E': (500, 3000)}
    dividas = {'A/B': (1000, 5000), 'C': (500, 2000), 'D/E': (0, 500)}
    
    df_clientes['limite_cartao'] = df_clientes['classe_social'].apply(lambda x: np.random.uniform(*limites[x]))
    df_clientes['scr_divida_inicial'] = df_clientes['classe_social'].apply(lambda x: np.random.uniform(*dividas[x]))

    # =============================================================================
    # 🎭 GRUPO DE SIMULAÇÃO - O CORAÇÃO DO CASE
    # =============================================================================
    def definir_grupo(row):
        # Clientes "instáveis" (Perfil Sem Reserva ou Economiza)
        if row['perfil_investidor'] in ['Sem Reserva', 'Economiza']:
            return 'instavel'
        # Clientes "estáveis" (Perfil Diversifica ou Caderneta)
        else:
            # Sortear uma parte para o grupo de estresse
            if np.random.rand() < PCT_ESTRESSE:
                return 'estavel_target'
            else:
                return 'estavel_controle'
                
    df_clientes['grupo_simulacao'] = df_clientes.apply(definir_grupo, axis=1)
    
    print("📊 Perfis estáticos criados:")
    print(df_clientes['grupo_simulacao'].value_counts(normalize=True))
    
    return df_clientes.set_index('id_cliente')

In [ ]:
# =============================================================================
# 📈 CARREGAR DADOS MACROECONÔMICOS (Banco Central)
# =============================================================================
def carregar_dados_macro():
    """
    🎯 OBJETIVO: Carregar dados de juros e inadimplência para contexto macroeconômico.
    """
    try:
        # Juros rotativo (20679), Juros pessoal (20665), Inadimplência PF (21084)
        df_rotativo = pd.read_csv('bcdata.sgs.20679.csv', sep=';', decimal=',', parse_dates=['data'], dayfirst=True)
        df_pessoal = pd.read_csv('bcdata.sgs.20665.csv', sep=';', decimal=',', parse_dates=['data'], dayfirst=True)
        df_inadimp = pd.read_csv('bcdata.sgs.21084.csv', sep=';', decimal=',', parse_dates=['data'], dayfirst=True)
        
        # Limpando e formatando os dados
        df_rotativo['taxa_juros_rotativo'] = df_rotativo['valor'] / 100
        df_pessoal['taxa_juros_pessoal'] = df_pessoal['valor'] / 100
        df_inadimp['taxa_inadimplencia_pf'] = df_inadimp['valor']
        
        # Criando base macro mensal consolidada
        df_macro = df_rotativo[['data', 'taxa_juros_rotativo']].merge(
            df_pessoal[['data', 'taxa_juros_pessoal']], on='data'
        ).merge(
            df_inadimp[['data', 'taxa_inadimplencia_pf']], on='data'
        )
        
        # Criando 'mes_id' para merge com simulação
        df_macro = df_macro.sort_values('data').reset_index(drop=True)
        df_macro['mes_id'] = (df_macro['data'].dt.year - df_macro['data'].dt.year.min()) * 12 + df_macro['data'].dt.month
        
        # Pegando apenas os N_MESES mais recentes
        df_macro = df_macro.tail(N_MESES).reset_index(drop=True)
        df_macro['mes_simulacao'] = np.arange(1, N_MESES + 1)
        
        print(f\"📈 Dados macro carregados. Último mês: {df_macro['data'].max().date()}\")
        return df_macro[['mes_simulacao', 'taxa_juros_rotativo', 'taxa_juros_pessoal', 'taxa_inadimplencia_pf']]

    except FileNotFoundError:
        print(\"⚠️ Arquivos CSV do Banco Central não encontrados. Rodando sem dados macro.\")
        return None

In [ ]:
# =============================================================================
# 📅 FUNÇÃO PARA GERAR O HISTÓRICO COMPORTAMENTAL MÊS A MÊS
# =============================================================================
def gerar_historico_longitudinal(df_clientes, df_macro):
    """
    🎯 OBJETIVO: Criar histórico de transações mês a mês para cada cliente,
    simulando comportamento específico de cada 'grupo_simulacao'.
    """
    historico_total = []
    
    # Datas da simulação (últimos N_MESES meses)
    datas_mes = pd.date_range(end=datetime.date.today(), periods=N_MESES, freq='MS')
    
    for id_cliente, perfil in df_clientes.iterrows():
        
        # Parâmetros do cliente
        grupo = perfil['grupo_simulacao']
        limite = perfil['limite_cartao']
        divida_scr_atual = perfil['scr_divida_inicial']
        
        for mes_num in range(1, N_MESES + 1):
            
            # --- Parâmetros base de simulação ---
            gasto_cartao = 0
            pagamento_fatura = 0
            flag_rotativo = False
            
            # =============================================================================
            # 🎭 LÓGICA DE COMPORTAMENTO POR GRUPO
            # =============================================================================
            
            if grupo == 'instavel':
                # 🚨 Cliente \"ruim\": gasta muito e paga parcial aleatoriamente
                gasto_cartao = limite * np.random.uniform(0.5, 1.0)
                if np.random.rand() < 0.3:  # 30% de chance de pagar parcial
                    pagamento_fatura = gasto_cartao * np.random.uniform(0.15, 0.5)
                    flag_rotativo = True
                else:
                    pagamento_fatura = gasto_cartao
                # Dívida SCR cresce erraticamente
                divida_scr_atual *= np.random.uniform(0.95, 1.1)

            elif grupo == 'estavel_controle':
                # ✅ Cliente \"bom\" (controle): gasta pouco e paga integral
                gasto_cartao = limite * np.random.uniform(0.2, 0.5)
                pagamento_fatura = gasto_cartao
                flag_rotativo = False
                # Dívida SCR controlada
                divida_scr_atual *= np.random.uniform(0.98, 1.02)
                
            elif grupo == 'estavel_target':
                # 🎯 Cliente \"bom\" (alvo): se comporta bem ATÉ o estresse
                if mes_num < MES_INICIO_ESTRESSE:
                    # Comportamento normal
                    gasto_cartao = limite * np.random.uniform(0.2, 0.5)
                    pagamento_fatura = gasto_cartao
                    flag_rotativo = False
                    divida_scr_atual *= np.random.uniform(0.98, 1.02)
                else:
                    # ⚠️ EVENTO DE ESTRESSE (AS FEATURES QUE O MODELO DEVE CAPTAR!)
                    # 1. Aumento do uso do cartão
                    gasto_cartao = limite * np.random.uniform(0.7, 1.1)
                    # 2. Aumento do endividamento no SCR
                    divida_scr_atual *= np.random.uniform(1.1, 1.4)
                    
                    # CONSEQUÊNCIA: Cliente não consegue pagar integralmente
                    pagamento_fatura = gasto_cartao * np.random.uniform(0.15, 0.3)
                    flag_rotativo = True
            
            # Adiciona o registro mensal do cliente
            historico_total.append({
                'id_cliente': id_cliente,
                'mes_referencia': datas_mes[mes_num-1],
                'mes_simulacao': mes_num,
                'gasto_total_cartao': gasto_cartao,
                'pagamento_fatura': pagamento_fatura,
                'utilizacao_limite_cartao': gasto_cartao / limite,
                'flag_rotativo': flag_rotativo,
                'scr_saldo_devedor_total': divida_scr_atual
            })
            
    df_hist = pd.DataFrame(historico_total)
    
    # Juntar dados macroeconômicos ao histórico (se disponíveis)
    if df_macro is not None:
        df_hist = df_hist.merge(df_macro, on='mes_simulacao', how='left')
        
    return df_hist.merge(df_clientes, on='id_cliente', how='left')

In [ ]:
# =============================================================================
# 📊 FUNÇÃO PARA CRIAR AS FEATURES DE JANELA MÓVEL (ENGENHARIA DE FEATURES)
# =============================================================================
def criar_features_janeladas(df):
    """
    🎯 OBJETIVO: Criar features de tendência (últimos 3 meses) que o modelo usará.
    
    💡 INSIGHTS DE NEGÓCIO IMPLEMENTADOS:
    1. Crescimento dos gastos no cartão (último mês vs média 3 meses)
    2. Crescimento da dívida no SCR (último mês vs média 3 meses)
    3. Utilização média do limite (3 meses)
    4. Frequência de uso do rotativo (3 meses)
    """
    print(\"📊 Criando features de janela móvel (tendência)...\")
    
    # Ordenar o DataFrame
    df = df.sort_values(['id_cliente', 'mes_referencia'])
    
    # Configurar agrupamento por cliente
    g = df.groupby('id_cliente')
    
    # =============================================================================
    # 📈 CÁLCULO DAS FEATURES DE TENDÊNCIA
    # =============================================================================
    
    # 1. 🎯 \"Aumento no uso do cartão de crédito\" (Feature Principal)
    gasto_media_3m = g['gasto_total_cartao'].rolling(window=3, min_periods=1).mean().shift(1).reset_index(level=0, drop=True)
    gasto_ult_mes = g['gasto_total_cartao'].shift(1).reset_index(level=0, drop=True)
    
    df['gasto_crescim_3m'] = (gasto_ult_mes / gasto_media_3m) - 1
    
    # 2. 🎯 \"Aumento do endividamento no SCR\" (Feature Principal)
    scr_media_3m = g['scr_saldo_devedor_total'].rolling(window=3, min_periods=1).mean().shift(1).reset_index(level=0, drop=True)
    scr_ult_mes = g['scr_saldo_devedor_total'].shift(1).reset_index(level=0, drop=True)
    
    df['scr_crescim_divida_3m'] = (scr_ult_mes / scr_media_3m) - 1
    
    # --- Outras features de suporte importantes ---
    df['utilizacao_limite_media_3m'] = g['utilizacao_limite_cartao'].rolling(window=3, min_periods=1).mean().shift(1).reset_index(level=0, drop=True)
    df['contagem_rotativo_3m'] = g['flag_rotativo'].rolling(window=3, min_periods=1).sum().shift(1).reset_index(level=0, drop=True)

    # Preenchendo NaNs e corrigindo infinitos
    df = df.fillna(0)
    df = df.replace([np.inf, -np.inf], 0)
    
    return df

In [ ]:
# =============================================================================
# 🎯 FUNÇÃO PARA CRIAR A VARIÁVEL ALVO (TARGET)
# =============================================================================
def criar_variavel_alvo(df):
    """
    🎯 OBJETIVO: Criar a coluna 'target' que o modelo de ML tentará prever.
    
    📝 DEFINIÇÃO DO TARGET: 
    \"O cliente VAI entrar no rotativo no PRÓXIMO mês?\"
    """
    print(\"🎯 Criando variável alvo...\")
    df = df.sort_values(['id_cliente', 'mes_referencia'])
    
    # TARGET: Usamos shift(-1) para trazer o futuro (\"próximo mês\") para a linha atual
    df['entrou_rotativo_proximo_mes'] = df.groupby('id_cliente')['flag_rotativo'].shift(-1)
    
    # Removemos o último mês de cada cliente (não temos \"próximo\" mês)
    df_ml = df.dropna(subset=['entrou_rotativo_proximo_mes'])
    
    # Convertendo para 0/1 (classificação binária)
    df_ml['entrou_rotativo_proximo_mes'] = df_ml['entrou_rotativo_proximo_mes'].astype(int)
    
    return df_ml

In [ ]:
# =============================================================================
# 🚀 FUNÇÃO DE ORQUESTRAÇÃO PRINCIPAL
# =============================================================================
def main():
    """
    🎯 FLUXO PRINCIPAL DA SIMULAÇÃO
    1. Criar perfis estáticos dos clientes
    2. Carregar dados macroeconômicos 
    3. Gerar histórico comportamental mês a mês
    4. Criar features de tendência
    5. Criar variável alvo (target)
    6. Salvar base final para treinamento
    """
 
    # Passo 1: Criar perfis estáticos
    df_clientes = criar_clientes_estaticos(N_CLIENTES)
 
    # Passo 2: Carregar dados macro
    df_macro = carregar_dados_macro()

    # Passo 3: Gerar histórico comportamental
    df_historico_completo = gerar_historico_longitudinal(df_clientes, df_macro)

    # Passo 4: Criar features de tendência
    df_features = criar_features_janeladas(df_historico_completo)

    # Passo 5: Criar a variável alvo
    df_base_ml = criar_variavel_alvo(df_features)

    # =============================================================================
    # 💾 SALVAR A BASE FINAL PARA TREINAMENTO
    # =============================================================================
    print(f\"\\n💾 Salvando base de ML final...\")
    try:
        df_base_ml.to_parquet('base_simulada_clientes.parquet', index=False)
        print(\"✅ Sucesso! Base salva em 'base_simulada_clientes.parquet'\")
    except ImportError:
        df_base_ml.to_csv('base_simulada_clientes.csv', index=False, sep=';', decimal=','